# To upload groceries.csv

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving groceries.csv to groceries.csv
User uploaded file "groceries.csv" with length 510697 bytes


In [0]:
from google.colab import files
# files.download('c0.03.csv') 
files.download('c0.06.csv') 
files.download('c0.09.csv') 
files.download('s0.05.csv') 


In [33]:
!ls

c0.03.csv  c0.06.csv  c0.09.csv  groceries.csv	s0.05.csv  sample_data


# Install Anaconda

In [0]:
!wget -c https://repo.continuum.io/archive/Anaconda2-5.3.1-Linux-x86_64.sh
!chmod +x Anaconda2-5.3.1-Linux-x86_64.sh
!bash ./Anaconda2-5.3.1-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

# Code

Run code in the code cells <br>
The variables declared will be global i.e across all cells
Make new code cells

In [31]:
#Apriori
# coding: utf-8

# In[7]:


import pandas as pd
import numpy as np
import csv
import itertools
from itertools import chain,combinations


# In[185]:


min_support = 0.09
min_confidence = 0.005
transactions = list()
L1 = set()
with open('groceries.csv',newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        transactions.append(set(row))
        for item in row:
            L1.add(frozenset([item]))


# In[186]:


def pruned_itemset(global_support,itemset,minSupport,transactions):
    set1 = set() #contains pruned set of items
    supcount = dict() #stores support count for those pruned itemsets
    for item in itemset:
        for _ in transactions:
            if(set(item) <= set(_)):
                if(item in supcount.keys()):
                    supcount[item] += 1
                else: supcount[item] = 1
                if(item in global_support.keys()):
                    global_support[item] += 1
                else: global_support[item] = 1
    for item,sup_count in supcount.items():
        if(sup_count / len(transactions) >= minSupport):
            set1.add(item)
    return set1


# In[187]:


def self_join(Lset,k): #Here Lset is L(k-1), and Lset_k is L(k+1)
    join = []
    for set1 in Lset:
        for set2 in Lset:
            a = set1.union(set2)
            if(len(a) == k):
                join.append(a)
    return(set(join))


# In[188]:


def Apriori(minSupport):
    freq_itemsets = dict() #used to generate rules
    all_itemset_support = dict()
    k = 1
    C = pruned_itemset(all_itemset_support,L1,min_support,transactions)
    L = C
    freq_itemsets[k] = L
    k = k + 1
    while(L != set()):
        L = self_join(L,k)
        C = pruned_itemset(all_itemset_support,L,min_support,transactions)
        L = C
        freq_itemsets[k] = L
        k = k + 1    
    return freq_itemsets,all_itemset_support


# In[189]:


a,b = Apriori(min_support)


# In[190]:


def gen_rules(min_confidence,a,b):
    rules = []
    c = list(a[2])
    k = 2
    while(c != list()):
        for each in c:
            temp = list(each)
            u = chain(*[combinations(temp, i + 1) for i in range(len(temp))])
            q = [x for x in u]
            for _ in q:
                p = frozenset(_)
                diff = each.difference(p)
                if(len(diff) > 0):
                    conf = b[each] / b[p]
                    if(conf >= min_confidence):
                        rules.append([p,diff,conf])
        k = k + 1
        c = list(a[k])
    return rules


# In[191]:


c = gen_rules(min_confidence,a,b)


# In[192]:


F = []
q = a[1]
k = 1
while(q != set()):
    for each in q:
        F.append(each)
    k = k + 1
    q = a[k]


# In[140]:


with open('s0.05.csv','a') as file:
    writer = csv.writer(file)
    for _ in F:
        row = [list(_),b[_]]
        print(row)
        writer.writerow(row)


# In[193]:


with open('c0.09.csv','a') as file:
    writer = csv.writer(file)
    for _ in c:
        row = [list(map(list,_[0:-1])),_[-1]]
        writer.writerow(row)


# In[ ]:






[['other vegetables'], 1903]
[['soda'], 1715]
[['whole milk'], 2513]
[['tropical fruit'], 1032]
[['shopping bags'], 969]
[['bottled water'], 1087]
[['yogurt'], 1372]
[['rolls/buns'], 1809]
[['root vegetables'], 1072]
[['sausage'], 924]


In [32]:
import pandas as pd
dfx = pd.read_csv('c0.09.csv')
print(dfx)

EmptyDataError: ignored

In [5]:
!ls

sample_data


In [0]:
import csv

min_support = 0.01
min_confidence = 0.5
transactions = list()
L1 = set()
with open('groceries.csv',newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        transactions.append(set(row))
        for item in row:
            L1.add(frozenset([item])) 

def pruned_itemset(itemset,minSupport,transactions):
    set1 = set() #contains pruned set of items
    supcount = dict() #stores support count for those pruned itemsets
    for item in itemset:
        for _ in transactions:
            if(set(item) <= set(_)):
                if(item in supcount.keys()):
                    supcount[item] += 1
                else: supcount[item] = 1
    for item,sup_count in supcount.items():
        if(float(sup_count / len(transactions)) >= minSupport):
            set1.add(item)
    return set1

def self_join(Lset,k): #Here Lset is L(k-1), and Lset_k is L(k+1)
    join = []
    for set1 in Lset:
        for set2 in Lset:
            a = set1.union(set2)
            if(len(a) == k):
                join.append(a)
    return(set(join))
  
def Apriori(minSupport,minConfidence):
    freq_itemsets = dict() #used to generate rules
    k = 1
    C = pruned_itemset(L1,min_support,transactions)
    L = C
    freq_itemsets[k] = L
    k = k + 1
    while(L != set()):
        L = self_join(L,k)
        C = pruned_itemset(L,min_support,transactions)
        L = C
        freq_itemsets[k] = L
        k = k + 1    
    return freq_itemsets
  
a = Apriori(min_support,min_confidence)

In [0]:
for item in a.values():
  print(item)

{frozenset({'mustard'}), frozenset({'cat food'}), frozenset({'potted plants'}), frozenset({'beef'}), frozenset({'chewing gum'}), frozenset({'other vegetables'}), frozenset({'semi-finished bread'}), frozenset({'butter'}), frozenset({'salty snack'}), frozenset({'liquor'}), frozenset({'newspapers'}), frozenset({'hygiene articles'}), frozenset({'chocolate'}), frozenset({'specialty bar'}), frozenset({'UHT-milk'}), frozenset({'hard cheese'}), frozenset({'shopping bags'}), frozenset({'butter milk'}), frozenset({'cake bar'}), frozenset({'seasonal products'}), frozenset({'sugar'}), frozenset({'curd'}), frozenset({'root vegetables'}), frozenset({'chicken'}), frozenset({'ice cream'}), frozenset({'pip fruit'}), frozenset({'cream cheese'}), frozenset({'roll products'}), frozenset({'long life bakery product'}), frozenset({'coffee'}), frozenset({'beverages'}), frozenset({'oil'}), frozenset({'white wine'}), frozenset({'dessert'}), frozenset({'meat'}), frozenset({'fruit/vegetable juice'}), frozenset({'

In [0]:
class FPNode(object):
    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None

    def add(self, child):
        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        """
        Check whether this node contains a child node for the given item.
        If so, that node is returned; otherwise, `None` is returned.
        """
        try:
            return self._children[item]
        except KeyError:
            return None

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        """The tree in which this node appears."""
        return self._tree

    @property
    def item(self):
        """The item contained in this node."""
        return self._item

    @property
    def count(self):
        """The count associated with this node's item."""
        return self._count

    def increment(self):
        """Increment the count associated with this node's item."""
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        """True if this node is the root of a tree; false if otherwise."""
        return self._item is None and self._count is None

    @property
    def leaf(self):
        """True if this node is a leaf in the tree; false if otherwise."""
        return len(self._children) == 0

    @property
    def parent(self):
        """The node's parent"""
        return self._parent

    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value

    @property
    def neighbor(self):
        """
        The node's neighbor; the one with the same value that is "to the right"
        of it in the tree.
        """
        return self._neighbor

    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value

    @property
    def children(self):
        """The nodes that are children of this node."""
        return tuple(self._children.values())

    def inspect(self, depth=0):
        print ('  ' * depth) + repr(self)
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)



In [0]:
class FPTree(object):

    Route = namedtuple('Route', 'head tail')

    def __init__(self):
        # The root node of the tree.
        self._root = FPNode(self, None, None)

        # A dictionary mapping items to the head and tail of a path of
        # "neighbors" that will hit every node containing that item.
        self._routes = {}

    @property
    def root(self):
        """The root node of the tree."""
        return self._root

    def add(self, transaction):
        """Add a transaction to the tree."""
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # There is already a node in this tree for the current
                # transaction item; reuse it.
                next_point.increment()
            else:
                # Create a new point and add it as a child of the point we're
                # currently looking at.
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the route of nodes that contain this item to include
                # our new node.
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        """Add the given node to the route through all nodes for its item."""
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # First node for this item; start a new route.
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        """
        Generate one 2-tuples for each item represented in the tree. The first
        element of the tuple is the item itself, and the second element is a
        generator that will yield the nodes in the tree that belong to the item.
        """
        for item in self._routes.iterkeys():
            yield (item, self.nodes(item))

    def nodes(self, item):
        """
        Generate the sequence of nodes that contain the given item.
        """

        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
            node = node.neighbor

    def prefix_paths(self, item):
        """Generate the prefix paths that end with the given item."""

        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        print 'Tree:'
        self.root.inspect(1)

        print
        print 'Routes:'
        for item, nodes in self.items():
            print '  %r' % item
            for node in nodes:
                print '    %r' % node

In [0]:
min_support = 0.01
min_confidence = 0.5
support_count = {}
for transaction in transactions:
  for item in transaction:
    if(item in support_count):
      support_count[item] += 1
    else:
      support_count[item] = 1
      
for key in list(support_count):
    if(support_count[key] < min_support * len(transactions)):
      del support_count[key]
      
support_count = sorted( ((v,k) for k,v in support_count.items()), reverse=True)

transaction_list = []

for i in range(len(transactions)):
  for j in list(transactions[i]):
    temp = []
    if j in support_count:
      temp.append(j)
#     temp = [x for _, x in sorted(zip(support_count,temp), key=lambda pair: pair[0])]
    transaction_list.append(temp)  


 